In [1]:
!pip install yfinance

In [184]:
import numpy as np
import pandas as pd
import yfinance as yf

In [83]:
#ALGORITHM OUTLINE
# (1) Find current_price for stock of interest (use stocks most recent close price)
# (2) Check if .93*current_price \in [low,high] for stock of interest for the last week (last five days)
# (3)

In [70]:
# algorithm for single date
# dat = yf.Ticker("^TASI.SR")
# history_for_lastweek_5d = dat.history(period='5d')
# low = history_for_lastweek_5d["Low"].min()
# high = history_for_lastweek_5d["High"].max()
# current_price = history_for_lastweek_5d["Close"][0]
# price_of_interest = .93*current_price
# price_of_interest_within_last_week = low <= price_of_interest and price_of_interest <= high

<ipython-input-70-2770be6dda2d>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = history_for_lastweek_5d["Close"][0]


In [309]:
# algorithm for every date between start date and end date
data = yf.download("^TASI.SR", start="2010-01-01", end="2025-01-23")

data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)

data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)

data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data.tail(5)

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,seven_percent_of_close_price_within_last_five_days,up_from_yesterday,buy,Close_up1%,Close_down1%
Ticker,^TASI.SR,^TASI.SR,^TASI.SR,^TASI.SR,^TASI.SR,,,,,,,
Date,,,,,,,,,,,,
2025-01-16,12256.059570,12262.799805,12219.490234,12236.519531,326466400,12041.349609,12222.559570,False,True,False,NaN,NaN
2025-01-19,12331.870117,12346.820312,12259.309570,12264.379883,370535800,12101.450195,12262.799805,False,True,False,NaN,NaN
2025-01-20,12379.540039,12398.839844,12335.440430,12336.509766,278757700,12101.450195,12346.820312,False,True,False,NaN,NaN
2025-01-21,12369.629883,12402.099609,12341.839844,12393.370117,293385400,12109.639648,12398.839844,False,False,False,NaN,NaN
2025-01-22,12362.389648,12434.540039,12362.389648,12376.379883,346099500,12169.440430,12402.099609,False,False,False,NaN,NaN


In [310]:
data.loc[data.buy == True].head(5)

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,seven_percent_of_close_price_within_last_five_days,up_from_yesterday,buy,Close_up1%,Close_down1%
Ticker,^TASI.SR,^TASI.SR,^TASI.SR,^TASI.SR,^TASI.SR,,,,,,,
Date,,,,,,,,,,,,
2010-06-14,6340.029785,6345.810059,6257.850098,6262.560059,154124500,5873.089844,6114.589844,True,True,True,6403.430083,6276.629487
2011-03-07,5950.740234,5952.990234,5678.229980,5762.500000,210974100,5231.509766,6263.790039,True,True,True,6010.247637,5891.232832
2011-03-08,6073.790039,6157.919922,5897.049805,5950.740234,308367700,5231.509766,5995.370117,True,True,True,6134.527939,6013.052139
2011-03-09,6108.669922,6157.970215,6073.669922,6073.790039,242729300,5231.509766,6157.919922,True,True,True,6169.756621,6047.583223
2011-03-14,6230.700195,6309.629883,6185.890137,6309.629883,307629600,5323.270020,6157.970215,True,True,True,6293.007197,6168.393193
